In [2]:
import pandas as pd
from urllib.parse import urlparse, unquote

# Load the CSV file
df = pd.read_csv("essex_url_hierarchy_final.csv")

# Initialize dictionary to hold dictionaries of URLs per level_1
url_dicts = {}

# Get unique level_1 values from the CSV
unique_level_1_values = df['level_1'].dropna().unique()

# Iterate through each unique level_1 value to create URL dictionaries
for level in unique_level_1_values:
    urls = df.loc[df['level_1'] == level, 'url'].dropna()
    level_dict = {}
    
    for u in urls:
        parts = unquote(urlparse(u).path).rstrip("/").split("/")
        try:
            level_idx = parts.index(level)
            key_parts = parts[level_idx + 1:]
            key = "/".join(key_parts) if key_parts else level
        except ValueError:
            key = "/".join(parts)
        
        level_dict[key] = u
    
    url_dicts[level] = level_dict

# Print number of entries per level_1 category
dict_summary = {level: len(url_dicts[level]) for level in url_dicts}
print("\n--- Summary of All Dictionaries ---")
for level, count in dict_summary.items():
    print(f"{level}: {count}")

# Print all key-value pairs in a dictionary
print("\n--- See Dictionary ---")
for k, v in url_dicts['alumni'].items():
    print(f"{k}: {v}")


--- Summary of All Dictionaries ---
research: 78
blog: 13
life: 86
student: 677
staff: 936
centres-and-institutes: 209
sport: 55
about: 39
test: 20
departments: 562
governance-and-strategy: 49
postgraduate: 62
wivenhoe-park: 4
event-series: 15
jobs: 16
global: 12
business: 45
schools-and-colleges: 44
alumni: 38
sustainability: 12
study-online: 1
choir: 6
visit-us: 11
china: 6
undergraduate: 21
donate: 9
apprenticeships: 3
study-abroad: 25
graduation: 14
scholarships: 60
welcome: 17
international: 108
short-courses: 91
events: 31
research-projects: 363
disclaimer: 14
sport-homepage-test: 1
clearing: 7
arena: 5
fees-and-funding: 1
subjects: 61
news: 2537
people: 2015
courses: 685

--- See Dictionary ---
alumni: https://www.essex.ac.uk/alumni
awards/alumnus-of-the-year/2016: https://www.essex.ac.uk/alumni/awards/alumnus-of-the-year/2016
awards/alumnus-of-the-year/2011: https://www.essex.ac.uk/alumni/awards/alumnus-of-the-year/2011
awards/alumnus-of-the-year/2008: https://www.essex.ac.uk/

In [4]:
def urls_level_1_cuter(dict_name):
    """
    Filters URLs from a specified level_1 category dictionary, skipping URLs that only contain 
    the base category path (1 section after the domain).

    Args:
        dict_name (str): Name of the level_1 category dictionary (e.g., "subjects")

    Returns:
        dict: Filtered URLs with nested paths (>=2 sections after domain)
    """
    original_dict = url_dicts.get(dict_name, {})
    filtered_dict = {}
    
    for key, url in original_dict.items():
        parsed_url = urlparse(url)
        path_sections = [p for p in parsed_url.path.split("/") if p]
        
        # Keep only URLs with paths longer than base category
        # Example: Keep "/subjects/accounting" (2 sections), skip "/subjects" (1 section)
        if len(path_sections) > 1:
            filtered_dict[key] = url
            
    return filtered_dict

In [6]:
# Get cleaned URLs for "subjects" category
filtered_urls = urls_level_1_cuter("courses")

print(f"Cleaned URLs for 'subjects': {len(filtered_urls)}")
for key, url in filtered_urls.items():
    print(f"{key}: {url}")

Cleaned URLs for 'subjects': 685
UG00001/1/BSc-Accounting: https://www.essex.ac.uk/courses/UG00001/1/BSc-Accounting
PG00425/1/MSc-Accounting: https://www.essex.ac.uk/courses/PG00425/1/MSc-Accounting
PR00854/2/PhD-Accounting: https://www.essex.ac.uk/courses/PR00854/2/PhD-Accounting
UG00002/1/BSc-Accounting-and-Finance: https://www.essex.ac.uk/courses/UG00002/1/BSc-Accounting-and-Finance
PR00855/5/PhD-Accounting-and-Finance: https://www.essex.ac.uk/courses/PR00855/5/PhD-Accounting-and-Finance
PR00854/1/PhD-Accounting: https://www.essex.ac.uk/courses/PR00854/1/PhD-Accounting
UG00002/2/BSc-Accounting-and-Finance: https://www.essex.ac.uk/courses/UG00002/2/BSc-Accounting-and-Finance
UG00002/3/Integrated-Master-in-Accounting-Accounting-and-Finance: https://www.essex.ac.uk/courses/UG00002/3/Integrated-Master-in-Accounting-Accounting-and-Finance
PG00425/4/MRes-Accounting: https://www.essex.ac.uk/courses/PG00425/4/MRes-Accounting
PR00855/1/PhD-Accounting-and-Finance: https://www.essex.ac.uk/cour

In [ ]:
# # ╔══════════════════════════════════════════════════════════════╗
# # ║  batch_scrape_to_markdown - Text Only Version               ║
# # ╚══════════════════════════════════════════════════════════════╝
# import asyncio, os, re, time
# from pathlib import Path
# from urllib.parse import urljoin, urlsplit, quote

# from bs4 import BeautifulSoup
# from markdownify import markdownify as md
# from tqdm.auto import tqdm

# from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
# from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

# # ─────────────────────────── noise filters ─────────────────────
# NOISE_RE   = re.compile(
#     r"(cookie|consent|banner|nav|menu|breadcrumb|footer|sidebar|masthead|navbar|"
#     r"pager|pagination|social|share)",
#     re.I,
# )
# STRIP_TAGS = {"svg", "style", "script", "img"}  # Added 'img' to strip images

# # ─────────────────────────── helpers ───────────────────────────
# def _best_wrapper(soup: BeautifulSoup):
#     for sel in ("main[id*='Main-Content' i]", "main", "article", "#content", ".content"):
#         node = soup.select_one(sel)
#         if node:
#             return node
#     divs = sorted(soup.find_all("div"),
#                   key=lambda d: len(d.get_text(" ", strip=True)),
#                   reverse=True)
#     return divs[0] if divs else soup.body or soup

# def _strip_noise(node: BeautifulSoup):
#     for tag in node.find_all(["header", "nav", "footer", "aside"]):
#         tag.decompose()
#     for tag in node.find_all(
#         lambda t: (t.has_attr("id") and NOISE_RE.search(t["id"]))
#         or (t.has_attr("class") and any(NOISE_RE.search(c) for c in t["class"]))
#     ):
#         tag.decompose()
#     for tag in node.find_all(STRIP_TAGS):
#         tag.decompose()

# # ─────────────────────────── single page scraper ──────────────
# async def scrape_single_page_text(url: str, css_selector: str | None = None) -> str:
#     """
#     Scrape a single page and return just the markdown text content.
#     """
#     BASE = f"{urlsplit(url).scheme}://{urlsplit(url).netloc}"

#     run_cfg = CrawlerRunConfig(
#         css_selector="body",
#         wait_for_images=False,  # No need to wait for images
#         remove_overlay_elements=True,
#         js_code="""(() => {
#             [...document.querySelectorAll('button,input[type="button"]')]
#               .filter(b => /accept|agree|ok/i.test(b.textContent))
#               .forEach(b => b.click());
#         })();""",
#         markdown_generator=DefaultMarkdownGenerator(
#             options={"ignore_links": False, "ignore_images": True}  # Ignore images
#         ),
#     )

#     async with AsyncWebCrawler() as crawler:
#         result = await crawler.arun(url, config=run_cfg)

#     if not result.success:
#         raise RuntimeError(f"Failed to scrape {url}: {result.error_message}")

#     soup = BeautifulSoup(result.html, "html.parser")
#     main = soup.select_one(css_selector) if css_selector else _best_wrapper(soup)
#     html = BeautifulSoup(str(main), "html.parser")

#     _strip_noise(html)

#     # ---- rewrite links (keep text links but remove image references) ---------
#     for a in html.find_all("a", href=True):
#         href = a["href"].strip()
#         if not href or href.startswith(("javascript:", "#")):
#             a.unwrap()
#             continue
#         if href.startswith("//"):
#             href = "https:" + href
#         elif not href.startswith(("http://", "https://", "mailto:", "tel:")):
#             href = urljoin(BASE, href)
#         a["href"] = quote(href, safe="/#:?=&;+@-._~%")

#     # ---- HTML → Markdown (text only) ------------------------------------------
#     body_md = md(str(html), heading_style="ATX", strip=["script", "style", "img"])
    
#     return body_md.strip()

# # ─────────────────────────── batch processor ───────────────────
# async def batch_scrape_to_markdown(
#     url_dict: dict[str, str],
#     *,
#     output_dir: str | Path = "scraped_pages",
#     css_selector: str | None = None,
#     max_concurrent: int = 5
# ) -> dict[str, Path]:
#     """
#     Batch scrape URLs from a dictionary and save as markdown files (text only).
    
#     Args:
#         url_dict: Dictionary with {key: url} pairs
#         output_dir: Directory to save markdown files
#         css_selector: Optional CSS selector for content extraction
#         max_concurrent: Maximum number of concurrent requests
    
#     Returns:
#         Dictionary mapping keys to saved file paths
#     """
#     start_time = time.time()
    
#     output_dir = Path(output_dir)
#     output_dir.mkdir(exist_ok=True)
    
#     results = {}
#     semaphore = asyncio.Semaphore(max_concurrent)
    
#     async def scrape_and_save(key: str, url: str) -> tuple[str, Path | None]:
#         async with semaphore:
#             try:
#                 print(f"📄 Scraping: {key}")
#                 content = await scrape_single_page_text(url, css_selector)
                
#                 # Create safe filename from key
#                 safe_filename = re.sub(r'[^\w\-_.]', '_', key)
#                 if not safe_filename.endswith('.md'):
#                     safe_filename += '.md'
                
#                 file_path = output_dir / safe_filename
                
#                 # Add YAML front-matter
#                 front_matter = f"---\nkey: {key}\npage_url: {url}\n---\n\n"
#                 full_content = front_matter + content
                
#                 file_path.write_text(full_content, encoding="utf-8")
#                 print(f"✅ Saved: {file_path}")
#                 return key, file_path
                
#             except Exception as e:
#                 print(f"❌ Error scraping {key} ({url}): {e}")
#                 return key, None
    
#     # Process all URLs concurrently
#     tasks = [scrape_and_save(key, url) for key, url in url_dict.items()]
    
#     print(f"🚀 Starting batch scrape of {len(url_dict)} URLs...")
#     with tqdm(total=len(tasks), desc="Scraping pages", unit="page") as pbar:
#         for coro in asyncio.as_completed(tasks):
#             key, file_path = await coro
#             results[key] = file_path
#             pbar.update(1)
    
#     end_time = time.time()
#     total_time = end_time - start_time
#     successful = sum(1 for path in results.values() if path is not None)
    
#     print(f"🎉 Completed! {successful}/{len(url_dict)} pages scraped successfully.")
#     print(f"⏱️  Total time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
#     if successful > 0:
#         print(f"📊 Average time per page: {total_time/successful:.2f} seconds")
    
#     return results


In [ ]:
# # ─────────────────────────── usage with your data ───────────────
# # To use with your filtered_urls:
# results = await batch_scrape_to_markdown(
#     filtered_urls, 
#     output_dir="business_content",
#     max_concurrent=1  # Adjust based on server limits
# )

In [8]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  batch_scrape_to_markdown - Text Only Version               ║
# ╚══════════════════════════════════════════════════════════════╝
import asyncio, os, re, time
from pathlib import Path
from urllib.parse import urljoin, urlsplit, quote, unquote

from bs4 import BeautifulSoup
from markdownify import markdownify as md
from tqdm.auto import tqdm

from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
# If the page does NOT need JavaScript, uncomment the line below for faster runs
from crawl4ai.async_crawler_strategy import AsyncHTTPCrawlerStrategy

# ─────────────────────────── noise filters ─────────────────────
NOISE_RE   = re.compile(
    r"(cookie|consent|banner|nav|menu|breadcrumb|footer|sidebar|masthead|navbar|"
    r"pager|pagination|social|share)",
    re.I,
)
STRIP_TAGS = {"svg", "style", "script", "img"}  # Added 'img' to strip images

# ─────────────────────────── helpers ───────────────────────────
def _safe_url(url: str) -> str:
    """Clean and properly encode URL to handle special characters."""
    # Parse the URL
    parts = urlsplit(url)
    
    # Re-encode the path to handle special characters like &
    safe_path = quote(unquote(parts.path), safe='/-')
    
    # Reconstruct the URL
    return f"{parts.scheme}://{parts.netloc}{safe_path}"

def _best_wrapper(soup: BeautifulSoup):
    for sel in ("main[id*='Main-Content' i]", "main", "article", "#content", ".content"):
        node = soup.select_one(sel)
        if node:
            return node
    divs = sorted(soup.find_all("div"),
                  key=lambda d: len(d.get_text(" ", strip=True)),
                  reverse=True)
    return divs[0] if divs else soup.body or soup

def _strip_noise(node: BeautifulSoup):
    for tag in node.find_all(["header", "nav", "footer", "aside"]):
        tag.decompose()
    for tag in node.find_all(
        lambda t: (t.has_attr("id") and NOISE_RE.search(t["id"]))
        or (t.has_attr("class") and any(NOISE_RE.search(c) for c in t["class"]))
    ):
        tag.decompose()
    for tag in node.find_all(STRIP_TAGS):
        tag.decompose()

# ─────────────────────────── single page scraper ──────────────
async def scrape_single_page_text(
    url: str, 
    css_selector: str | None = None,
    use_fast_mode: bool = False,
    fallback_to_standard: bool = True
) -> str:
    """
    Scrape a single page and return just the markdown text content.
    
    Args:
        url: URL to scrape
        css_selector: Optional CSS selector for content extraction
        use_fast_mode: Use HTTP-only strategy (faster but no JS support)
        fallback_to_standard: If fast mode fails, try standard mode
    """
    # Clean URL to handle special characters
    clean_url = _safe_url(url)
    BASE = f"{urlsplit(clean_url).scheme}://{urlsplit(clean_url).netloc}"

    run_cfg = CrawlerRunConfig(
        css_selector="body",
        wait_for_images=False,
        remove_overlay_elements=True,
        page_timeout=15000,
        js_code="""(() => {
            [...document.querySelectorAll('button,input[type="button"]')]
              .filter(b => /accept|agree|ok/i.test(b.textContent))
              .forEach(b => b.click());
        })();""" if not use_fast_mode else None,
        markdown_generator=DefaultMarkdownGenerator(
            options={"ignore_links": False, "ignore_images": True}
        ),
    )

    # Try fast mode first, then fallback to standard if needed
    result = None
    error_msg = ""
    
    if use_fast_mode:
        try:
            async with AsyncWebCrawler(crawler_strategy=AsyncHTTPCrawlerStrategy()) as crawler:
                result = await crawler.arun(clean_url, config=run_cfg)
        except Exception as e:
            error_msg = str(e)
            if not fallback_to_standard:
                raise RuntimeError(f"Fast mode failed for {url}: {error_msg}")
    
    # Use standard mode if fast mode failed or wasn't requested
    if result is None or not result.success:
        if use_fast_mode and fallback_to_standard:
            print(f"⚠️  Fast mode failed for {url}, trying standard mode...")
        
        async with AsyncWebCrawler() as crawler:
            result = await crawler.arun(clean_url, config=run_cfg)

    if not result.success:
        raise RuntimeError(f"Failed to scrape {url}: {result.error_message}")

    soup = BeautifulSoup(result.html, "html.parser")
    main = soup.select_one(css_selector) if css_selector else _best_wrapper(soup)
    html = BeautifulSoup(str(main), "html.parser")

    _strip_noise(html)

    # ---- rewrite links (keep text links but remove image references) ---------
    for a in html.find_all("a", href=True):
        href = a["href"].strip()
        if not href or href.startswith(("javascript:", "#")):
            a.unwrap()
            continue
        if href.startswith("//"):
            href = "https:" + href
        elif not href.startswith(("http://", "https://", "mailto:", "tel:")):
            href = urljoin(BASE, href)
        a["href"] = quote(href, safe="/#:?=&;+@-._~%")

    # ---- HTML → Markdown (text only) ------------------------------------------
    body_md = md(str(html), heading_style="ATX", strip=["script", "style", "img"])
    
    return body_md.strip()

# ─────────────────────────── batch processor ───────────────────
async def batch_scrape_to_markdown(
    url_dict: dict[str, str],
    *,
    output_dir: str | Path = "scraped_pages",
    css_selector: str | None = None,
    max_concurrent: int = 5,
    use_fast_mode: bool = False,
    delay_between_requests: float = 0.0
) -> dict[str, Path]:
    """
    Batch scrape URLs from a dictionary and save as markdown files (text only).
    
    Args:
        url_dict: Dictionary with {key: url} pairs
        output_dir: Directory to save markdown files
        css_selector: Optional CSS selector for content extraction
        max_concurrent: Maximum number of concurrent requests
        use_fast_mode: Use HTTP-only strategy (faster but no JS support)
        delay_between_requests: Delay in seconds between requests (for politeness)
    
    Returns:
        Dictionary mapping keys to saved file paths
    """
    start_time = time.time()
    
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    results = {}
    semaphore = asyncio.Semaphore(max_concurrent)
    
    async def scrape_and_save(key: str, url: str) -> tuple[str, Path | None]:
        async with semaphore:
            try:
                if delay_between_requests > 0:
                    await asyncio.sleep(delay_between_requests)
                
                print(f"📄 Scraping: {key}")
                content = await scrape_single_page_text(url, css_selector, use_fast_mode)
                
                # Create safe filename from key
                safe_filename = re.sub(r'[^\w\-_.]', '_', key)
                if not safe_filename.endswith('.md'):
                    safe_filename += '.md'
                
                file_path = output_dir / safe_filename
                
                # Add YAML front-matter
                front_matter = f"---\nkey: {key}\npage_url: {url}\nscraped_at: {time.strftime('%Y-%m-%d %H:%M:%S')}\n---\n\n"
                full_content = front_matter + content
                
                file_path.write_text(full_content, encoding="utf-8")
                print(f"✅ Saved: {file_path}")
                return key, file_path
                
            except Exception as e:
                print(f"❌ Error scraping {key} ({url}): {e}")
                return key, None
    
    # Process all URLs concurrently (or sequentially if max_concurrent=1)
    tasks = [scrape_and_save(key, url) for key, url in url_dict.items()]
    
    mode_str = "FAST MODE (HTTP-only)" if use_fast_mode else "STANDARD MODE (Playwright)"
    print(f"🚀 Starting batch scrape of {len(url_dict)} URLs in {mode_str}...")
    
    with tqdm(total=len(tasks), desc="Scraping pages", unit="page") as pbar:
        for coro in asyncio.as_completed(tasks):
            key, file_path = await coro
            results[key] = file_path
            pbar.update(1)
    
    end_time = time.time()
    total_time = end_time - start_time
    successful = sum(1 for path in results.values() if path is not None)
    
    print(f"🎉 Completed! {successful}/{len(url_dict)} pages scraped successfully.")
    print(f"⏱️  Total time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    if successful > 0:
        print(f"📊 Average time per page: {total_time/successful:.2f} seconds")
    
    return results



In [10]:
# ─────────────────────────── usage with your data ───────────────
# OPTION 1: Fast mode with automatic fallback (RECOMMENDED)
results = await batch_scrape_to_markdown(
    filtered_urls, 
    output_dir="courses_content",
    max_concurrent=1,
    use_fast_mode=True  # Will automatically fallback to standard mode if needed
)

# OPTION 2: Standard mode only (safe but slower)
# results = await batch_scrape_to_markdown(
#     filtered_urls, 
#     output_dir="business_content",
#     max_concurrent=1
# )

🚀 Starting batch scrape of 685 URLs in FAST MODE (HTTP-only)...


Scraping pages:   0%|          | 0/685 [00:00<?, ?page/s]

📄 Scraping: PG01644/1/MFA-Acting-with-Digital-Media
[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://www.essex.ac.uk/courses/PG01644/1/MFA-Acting-with-Digital-Media                              | ✓ | ⏱: 0.52s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PG01644/1/MFA-Acting-with-Digital-Media                              | ✓ | ⏱: 0.14s
[COMPLETE] ● https://www.essex.ac.uk/courses/PG01644/1/MFA-Acting-with-Digital-Media                              | ✓ | ⏱: 0.67s
✅ Saved: courses_content/PG01644_1_MFA-Acting-with-Digital-Media.md
📄 Scraping: PR00946/3/MSc-by-Dissertation-Microbiology
[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://www.essex.ac.uk/courses/PR00946/3/MSc-by-Dissertation-Microbiology                           | ✓ | ⏱: 0.60s
[SCRAPE].. ◆ https://www.essex.ac.uk/courses/PR00946/3/MSc-by-Dissertation-Microbiology                           | ✓ | ⏱: 0.15s
[COMPLETE] ● https://www.essex.ac.uk/courses/PR00946/3/MSc-by-Dissertation-Microbiology                           | ✓ | ⏱: 0